In [ ]:
%%capture
!pip install datasets
!pip install transformers==4.28.0
!pip install torchaudio
!pip install jiwer

In [ ]:
import zipfile
import os
import cv2 as cv
import numpy as np
import random
import pandas as pd
import IPython.display as ipd
import json
import re
import torch

from datasets import ClassLabel
from IPython.display import display, HTML
from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC
from datasets import load_dataset, load_metric, load_from_disk
from google.colab import output
from google.colab import drive
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


In [ ]:
#can skip this
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_PATH = '/content/drive/Shareddrives/C23-PR569/data/Dataset Ayat (bentuk array)/ArrayForm_dataset.zip'
zip_object = zipfile.ZipFile(file =DATASET_PATH ,mode = 'r')
zip_object.extractall('./')
zip_object.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/drive/Shareddrives/C23-PR569/data/Dataset Ayat (bentuk array)/ArrayForm_dataset.zip' mode='r'>>

In [ ]:
data = load_from_disk("/content/ArrayForm_dataset")
train_data = data["train"].select(range(4500))  # Select the first 4500 examples
test_data = data["test"].select(range(500))  # Select the first 500 examples

In [ ]:
def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_data, num_examples=5)

,path,sentence,speech,sampling_rate,target_text
0,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/082/010/082010_30.wav,وَإِنَّ عَلَيْكُمْ لَحَافِظِينَ,"[-1.1238633357280037e-09, -1.0785100590382513e-10, 1.0331557831477767e-09, 2.993425063380073e-09, 2.5270261438947728e-09, -4.5649756197363445e-10, -3.851196861415218e-10, -3.4770408774420503e-09, -2.6379973760981557e-09, 1.5799368480173825e-09, 5.512658662887304e-10, 1.88034832149242e-09, -2.345598382191838e-09, -4.5539052528909e-09, -3.193609376950235e-09, -3.5616274374206114e-09, -7.178669880758548e-10, 3.1166051961406538e-09, 4.790360996764775e-09, 7.566044457618659e-10, 7.122260559100368e-10, -1.001563720848253e-09, -2.2415387324059566e-09, -1.468065780052541e-09, 1.3769646534100843e-09, 2.3318922348636306e-09, 3.2108288250398687e-10, 2.7836255522828424e-10, 2.488675265865936e-09, 4.476928203156483e-11, 1.3123907516288114e-09, 1.5872134717653807e-09, 2.740191629158062e-09, 5.087307020090748e-09, 2.758107964240253e-09, -4.897167116268974e-10, 7.268590174192013e-10, -1.1001251021269809e-09, -7.428787585084251e-10, -7.252635159105125e-10, 4.1915582116303085e-10, -1.3959636779858897e-09, -2.713010260890769e-09, -3.7528575802525665e-09, -1.6191110674412812e-09, -1.8406784985103286e-09, -6.232560023633482e-10, 1.9069530399207224e-09, -3.5737290904336305e-10, 6.249942230418526e-10, 5.301425964887585e-10, 1.8643704358112245e-09, 1.7369496951857855e-09, 2.3810877713970058e-09, 2.781508801064092e-09, 8.109145577250843e-10, 2.8235425109102152e-09, -6.156258836043094e-10, 1.0609259026850282e-09, 1.0064687971933495e-10, 2.3825315054182283e-09, 1.8565158299566065e-09, 2.7540181246621387e-09, 2.832349021986147e-09, -1.008876537866854e-09, -1.0356326907157154e-09, -1.4397668612886605e-09, 1.5131440544990937e-09, 1.4230929767933276e-09, 7.717960714970218e-10, -3.569788464830026e-09, -6.247047323881816e-09, -3.85041065698033e-09, -4.8990891343692056e-09, -2.1601502808721307e-09, 1.0953586926376602e-09, 1.2795795534259469e-09, -3.6009040194073805e-09, -2.75574718600069e-09, -3.815876947754759e-09, -2.7218025611119856e-09, 3.8972058913344654e-10, -1.744578037587985e-09, -2.411428168258567e-09, 1.2181381459086538e-09, 6.84107326343053e-10, 2.581958424840991e-09, -7.502101717626886e-10, -2.314558766869368e-09, 7.616034469748456e-10, -1.2707725982608054e-09, -7.967051462998143e-10, 1.3094447748329685e-10, 7.074177910126878e-10, 3.354480915263025e-09, 1.490226053668664e-09, -2.622579930999791e-10, -2.819897426675766e-09, -2.626916018044767e-09, 1.077760103385117e-09, ...]",16000,وَإِنَّ عَلَيْكُمْ لَحَافِظِينَ
1,D:/Ar-DAD Arabic Diversified Audio Dataset/Ar-DAD Arabic Diversified Audio Dataset/Audio_Dataset/Reciters/082/014/082014_03.wav,وَإِنَّ الْفُجَّارَ لَفِي جَحِيمٍ,"[-5.479337414726615e-05, 1.459557461203076e-05, 0.00027961848536506295, 0.0006124371429905295, 0.0006873193196952343, 0.00012263037206139416, -0.00034785905154421926, -0.0001634947257116437, 5.24517017765902e-05, 0.00016985958791337907, 0.00019575300393626094, 0.00011984290176769719, 6.790637416997924e-05, 6.983913772273809e-05, 3.242919046897441e-05, 0.00015768606681376696, 0.00034296271041966975, 0.00014083384303376079, 7.512743468396366e-05, -5.336398317012936e-05, -0.00037486827932298183, -0.0002689099055714905, -0.00021480838768184185, -0.00022150709992274642, -0.00011444459232734516, -0.00042281649075448513, -0.0007129197474569082, -0.0006606398383155465, -0.00035264791222289205, -9.274626791011542e-05, 6.168869731482118e-05, -5.699553730664775e-05, -0.00026603491278365254, -0.0004063341475557536, -0.0003598544863052666, -0.00022628747683484107, -0.00011764658847823739, 2.6373189029982314e-05, -0.00018602680938784033, -0.00020962586859241128, -8.478119707433507e-05, -5.905806028749794e-05, -3.992950223619118e-05, 0.00021375203505158424, 0.00031048196251504123, 7.511588773922995e-05, 7.614798960275948e-05, 0.00020627916092053056, 0.00020008289720863104, 0.00035014146

In [ ]:
rand_int = random.randint(0, len(train_data))
ipd.Audio(data=np.asarray(train_data[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: أَلْهَاكُمُ التَّكَاثُرُ 
Input array shape: (53081,)
Sampling rate: 16000


In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
train_data = train_data.map(remove_special_characters)
test_data = test_data.map(remove_special_characters)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(train_data.remove_columns(["path", "speech", "sampling_rate","target_text"]), num_examples=10)

,sentence
0,إِذَا السَّمَاءُ انْشَقَّتْ
1,وَوُجُوهٌ يَوْمَئِذٍ عَلَيْهَا غَبَرَةٌ
2,أَلْهَاكُمُ التَّكَاثُرُ
3,كَلَّا إِنَّهَا تَذْكِرَةٌ
4,لَقَدْ خَلَقْنَا الْإِنْسَانَ فِي أَحْسَنِ تَقْوِيمٍ
5,عَلَّمَ الْإِنْسَانَ مَا لَمْ يَعْلَمْ
6,فَمَا يُكَذِّبُكَ بَعْدُ بِالدِّينِ
7,فَإِنَّ الْجَنَّةَ هِيَ الْمَأْوَى
8,وَإِنَّ الْفُجَّارَ لَفِي جَحِيمٍ
9,وَفُتِحَتِ السَّمَاءُ فَكَانَتْ أَبْوَابًا


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)
vocab_test = test_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_data.column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ة': 0,
 'س': 1,
 'آ': 2,
 'د': 3,
 'ص': 4,
 'ز': 5,
 'ك': 6,
 'غ': 7,
 'ي': 8,
 'ث': 9,
 'م': 10,
 'ع': 11,
 'ش': 12,
 'ى': 13,
 'ً': 14,
 'ط': 15,
 'ج': 16,
 'خ': 17,
 'ح': 18,
 'أ': 19,
 'ر': 20,
 'و': 21,
 'ٍ': 22,
 'ِ': 23,
 'ف': 24,
 'ض': 25,
 ' ': 26,
 'ن': 27,
 'ظ': 28,
 'إ': 29,
 'ت': 30,
 'ه': 31,
 'ٌ': 32,
 'ُ': 33,
 'ء': 34,
 'ذ': 35,
 'ؤ': 36,
 'ئ': 37,
 'ّ': 38,
 'َ': 39,
 'ْ': 40,
 'ب': 41,
 'ق': 42,
 'ل': 43,
 'ا': 44}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

47

In [ ]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "zarko1231/model-baru-collab-6" #nama repository huggingface untuk menyimpan model

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/zarko1231/model-baru-collab-6/commit/1a7322d45f8259d225de17a7cb871390f5bec6f1', commit_message='Upload tokenizer', commit_description='', oid='1a7322d45f8259d225de17a7cb871390f5bec6f1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer,)

In [ ]:
def prepare_dataset(batch):
    audio = batch

    batch["input_values"] = processor(audio["speech"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["sentence"], return_tensors="pt").input_ids[0]
    return batch

In [ ]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names,num_proc=4)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names,num_proc=4)

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
max_input_length_in_sec = 6.0
train_data = train_data.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
test_data = test_data.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-32-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "zarko1231/model-baru-collab",
    attention_dropout=0.3,
    hidden_dropout=0.2,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ignore_mismatched_sizes=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

In [ ]:
model.freeze_feature_encoder()

In [ ]:
print(model)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [ ]:
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=50,
  #auto_find_batch_size = True,
  gradient_accumulation_steps=2,
  logging_strategy='epoch',
  evaluation_strategy="epoch",
  num_train_epochs=25,
  gradient_checkpointing=True,
  fp16=True,
  learning_rate=2e-4,
  save_total_limit=1,
  save_strategy="epoch",
  push_to_hub=True,
  optim = "adamw_torch"
)


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Wer
1,3.822900,2.036937,1.047724
2,1.011200,0.161243,0.500000
3,0.225500,0.093763,0.366373
4,0.194500,0.092610,0.358297
5,0.132900,0.081145,0.342878
6,0.109500,0.079725,0.333333
7,0.103400,0.085686,0.331865
8,0.098700,0.079209,0.329662
9,0.106800,0.081487,0.327460
10,0.087500,0.081098,0.325257


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
trainer.push_to_hub(repo_name)

Upload file pytorch_model.bin:   0%|          | 1.00/1.18G [00:00<?, ?B/s]

Upload file runs/Jun06_03-54-47_96ea7db5b33c/events.out.tfevents.1686023783.96ea7db5b33c.537.0:   0%|         …

To https://huggingface.co/zarko1231/model-baru-collab-4
   d46ca2a..ce5ea1d  main -> main

   d46ca2a..ce5ea1d  main -> main

To https://huggingface.co/zarko1231/model-baru-collab-4
   ce5ea1d..180626e  main -> main

   ce5ea1d..180626e  main -> main



'https://huggingface.co/zarko1231/model-baru-collab-4/commit/ce5ea1d3ab3eb906615793403d6fae68e32f1a21'